## Import Library

In [ ]:
import pandas as pd
import numpy as np
import re
import string

## Load Dataset

In [ ]:
df = pd.read_csv("/content/Data_Rekomendasi_Jogja_fix.csv")
df.head()


,no,nama,vote_average,vote_count,type,htm_weekday,htm_weekend,latitude,longitude,description
0,9,Candi Borobudur,"4,7",81922,Budaya dan Sejarah,50000.0,50000.0,"-7,607086854","110,2036226",Candi yang pernah masuk sebagai salah satu dar...
1,10,Candi Prambanan,"4,7",71751,Budaya dan Sejarah,50000.0,50000.0,"-7,751834561","110,4915318",Candi Prambanan adalah kompleks candi Hindu te...
2,24,Tebing Breksi,"4,4",51431,Alam,10000.0,10000.0,"-7,781476547","110,5045757",Tebing Breksi merupakan tempat wisata yang ber...
3,343,Gembira Loka Zoo,"4,5",36337,Buatan,20000.0,25000.0,"-7,8062344","110,3967977",Gambira Loka adalah kebun binatang yang berada...
4,346,The Palace of Yogyakarta (Keraton Yogyakarta),"4,6",30091,Budaya dan Sejarah,8000.0,8000.0,"-7,8052845","110,3642031",Kompleks keraton merupakan museum yang menyimp...


## Cleaning Dataset

In [ ]:
print(df.isnull().sum())

no               0
nama             0
vote_average     0
vote_count       0
type             0
htm_weekday     27
htm_weekend     28
latitude         0
longitude        0
description      0
dtype: int64


In [ ]:
# Salin dataframe agar data mentah tetap aman
df_clean = df.copy()

# 1. Convert 'vote_average' from comma to dot and to float
df_clean['vote_average'] = df_clean['vote_average'].str.replace(',', '.', regex=False).astype(float)

# 2. Convert 'latitude' and 'longitude' from comma to dot and to float
df_clean['latitude'] = df_clean['latitude'].str.replace(',', '.', regex=False).astype(float)
df_clean['longitude'] = df_clean['longitude'].str.replace(',', '.', regex=False).astype(float)

# 3. Impute missing values in 'htm_weekday' and 'htm_weekend' with median per 'type'
df_clean['htm_weekday'] = df_clean.groupby('type')['htm_weekday'].transform(lambda x: x.fillna(x.median()))
df_clean['htm_weekend'] = df_clean.groupby('type')['htm_weekend'].transform(lambda x: x.fillna(x.median()))

# Check again for missing values
missing_summary = df_clean.isnull().sum()

df_clean.dtypes, missing_summary


(no                int64
 nama             object
 vote_average    float64
 vote_count        int64
 type             object
 htm_weekday     float64
 htm_weekend     float64
 latitude        float64
 longitude       float64
 description      object
 dtype: object,
 no              0
 nama            0
 vote_average    0
 vote_count      0
 type            0
 htm_weekday     0
 htm_weekend     0
 latitude        0
 longitude       0
 description     0
 dtype: int64)

In [ ]:
df_clean

,no,nama,vote_average,vote_count,type,htm_weekday,htm_weekend,latitude,longitude,description
0,9,Candi Borobudur,4.7,81922,Budaya dan Sejarah,50000.0,50000.0,-7.607087,110.203623,Candi yang pernah masuk sebagai salah satu dar...
1,10,Candi Prambanan,4.7,71751,Budaya dan Sejarah,50000.0,50000.0,-7.751835,110.491532,Candi Prambanan adalah kompleks candi Hindu te...
2,24,Tebing Breksi,4.4,51431,Alam,10000.0,10000.0,-7.781477,110.504576,Tebing Breksi merupakan tempat wisata yang ber...
3,343,Gembira Loka Zoo,4.5,36337,Buatan,20000.0,25000.0,-7.806234,110.396798,Gambira Loka adalah kebun binatang yang berada...
4,346,The Palace of Yogyakarta (Keraton Yogyakarta),4.6,30091,Budaya dan Sejarah,8000.0,8000.0,-7.805284,110.364203,Kompleks keraton merupakan museum yang menyimp...
...,...,...,...,...,...,...,...,...,...,...
471,139,Pemancingan Adi Winata,5.0,1,Wisata Air,0.0,0.0,-7.704577,110.512011,Tempat rekreasi keluarga yang menawarkan fasil...
472,105,Ruang Perawatan Jenderal Soedirman,5.0,1,Budaya dan Sejarah,4000.0,4000.0,-7.776474,110.376744,Situs sejarah berupa ruangan tempat Jenderal S...
473,110,Situs Gedong Pusoko,5.0,1,Budaya dan Sejarah,15000.0,15000.0,-7.807846,110.403758,Situs arkeologi yang menyimpan tinggalan sejar...
474,164,Taman Edukasi dan Outbound Sunan Kalijaga,5.0,1,Buatan,0.0,0.0,-7.809207,110.413252,Area edukatif yang menyediakan kegiatan outbou...


In [ ]:
# Fungsi untuk membersihkan
def clean_type(value):
    if pd.isna(value):
        return value
    value = value.strip().lower()

    replacements = {
        'argowisata': 'agrowisata',
        'agorwisata': 'agrowisata',
        'wisata air': 'wisata air',
        'wahana air': 'wisata air',
        'wIsata air': 'wisata air',
        'wisata air': 'wisata air',
        'budaya dan sejarahh': 'budaya dan sejarah',
        'museum,': 'museum'
    }

    # Normalisasi koma dan strip
    value = value.replace(" ,", ",").replace(", ", ",").replace(" , ", ",")

    # Pisahkan jika ada gabungan kategori
    parts = value.split(',')

    cleaned_parts = []
    for part in parts:
        part = part.strip()
        # Gunakan replacement jika tersedia
        cleaned = replacements.get(part, part)
        cleaned_parts.append(cleaned)

    # Ambil label utama pertama saja (bisa dikembangkan jadi multilabel)
    return cleaned_parts[0].title()  # Kapitalisasi pertama untuk konsistensi

# Terapkan ke kolom 'type'
df_clean['type_clean'] = df_clean['type'].apply(clean_type)

# Tampilkan nilai unik hasil pembersihan
df_clean['type_clean'].value_counts()


,count
type_clean,
Budaya Dan Sejarah,113
Alam,104
Agrowisata,70
Buatan,59
Pantai,51
Wisata Air,44
Museum,11
Religi,9
Kuliner,6


In [ ]:
df_clean.drop('type', axis=1, inplace=True)

In [ ]:
df_clean

,no,nama,vote_average,vote_count,htm_weekday,htm_weekend,latitude,longitude,description,type_clean
0,9,Candi Borobudur,4.7,81922,50000.0,50000.0,-7.607087,110.203623,Candi yang pernah masuk sebagai salah satu dar...,Budaya Dan Sejarah
1,10,Candi Prambanan,4.7,71751,50000.0,50000.0,-7.751835,110.491532,Candi Prambanan adalah kompleks candi Hindu te...,Budaya Dan Sejarah
2,24,Tebing Breksi,4.4,51431,10000.0,10000.0,-7.781477,110.504576,Tebing Breksi merupakan tempat wisata yang ber...,Alam
3,343,Gembira Loka Zoo,4.5,36337,20000.0,25000.0,-7.806234,110.396798,Gambira Loka adalah kebun binatang yang berada...,Buatan
4,346,The Palace of Yogyakarta (Keraton Yogyakarta),4.6,30091,8000.0,8000.0,-7.805284,110.364203,Kompleks keraton merupakan museum yang menyimp...,Budaya Dan Sejarah
...,...,...,...,...,...,...,...,...,...,...
471,139,Pemancingan Adi Winata,5.0,1,0.0,0.0,-7.704577,110.512011,Tempat rekreasi keluarga yang menawarkan fasil...,Wisata Air
472,105,Ruang Perawatan Jenderal Soedirman,5.0,1,4000.0,4000.0,-7.776474,110.376744,Situs sejarah berupa ruangan tempat Jenderal S...,Budaya Dan Sejarah
473,110,Situs Gedong Pusoko,5.0,1,15000.0,15000.0,-7.807846,110.403758,Situs arkeologi yang menyimpan tinggalan sejar...,Budaya Dan Sejarah
474,164,Taman Edukasi dan Outbound Sunan Kalijaga,5.0,1,0.0,0.0,-7.809207,110.413252,Area edukatif yang menyediakan kegiatan outbou...,Buatan


## Label Encoded

In [ ]:
df_encoded = df_clean.copy()

# Lakukan One-Hot Encoding
df_encoded = pd.get_dummies(df_clean, columns=['type_clean'])

# Lihat hasil
df_encoded

,no,nama,vote_average,vote_count,htm_weekday,htm_weekend,latitude,longitude,description,type_clean_Agrowisata,...,type_clean_Budaya Dan Sejarah,type_clean_Desa Wisata,type_clean_Kuliner,type_clean_Minat Khusus,type_clean_Museum,type_clean_Pantai,type_clean_Pendidikan,type_clean_Religi,type_clean_Seni,type_clean_Wisata Air
0,9,Candi Borobudur,4.7,81922,50000.0,50000.0,-7.607087,110.203623,Candi yang pernah masuk sebagai salah satu dar...,False,...,True,False,False,False,False,False,False,False,False,False
1,10,Candi Prambanan,4.7,71751,50000.0,50000.0,-7.751835,110.491532,Candi Prambanan adalah kompleks candi Hindu te...,False,...,True,False,False,False,False,False,False,False,False,False
2,24,Tebing Breksi,4.4,51431,10000.0,10000.0,-7.781477,110.504576,Tebing Breksi merupakan tempat wisata yang ber...,False,...,False,False,False,False,False,False,False,False,False,False
3,343,Gembira Loka Zoo,4.5,36337,20000.0,25000.0,-7.806234,110.396798,Gambira Loka adalah kebun binatang yang berada...,False,...,False,False,False,False,False,False,False,False,False,False
4,346,The Palace of Yogyakarta (Keraton Yogyakarta),4.6,30091,8000.0,8000.0,-7.805284,110.364203,Kompleks keraton merupakan museum yang menyimp...,False,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,139,Pemancingan Adi Winata,5.0,1,0.0,0.0,-7.704577,110.512011,Tempat rekreasi keluarga yang menawarkan fasil...,False,...,False,False,False,False,False,False,False,False,False,True
472,105,Ruang Perawatan Jenderal Soedirman,5.0,1,4000.0,4000.0,-7.776474,110.376744,Situs sejarah berupa ruangan tempat Jenderal S...,False,...,True,False,False,False,False,False,False,False,False,False
473,110,Situs Gedong Pusoko,5.0,1,15000.0,15000.0,-7.807846,110.403758,Situs arkeologi yang menyimpan tinggalan sejar...,False,...,True,False,False,False,False,False,False,False,False,False
474,164,Taman Edukasi dan Outbound Sunan Kalijaga,5.0,1,0.0,0.0,-7.809207,110.413252,Area edukatif yang menyediakan kegiatan outbou...,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# Cari semua kolom yang diawali 'type_clean_'
type_columns = [col for col in df_encoded.columns if col.startswith('type_clean_')]

# Ubah hanya kolom tersebut ke int
df_encoded[type_columns] = df_encoded[type_columns].astype(int)


In [ ]:
df_encoded

,no,nama,vote_average,vote_count,htm_weekday,htm_weekend,latitude,longitude,description,type_clean_Agrowisata,...,type_clean_Budaya Dan Sejarah,type_clean_Desa Wisata,type_clean_Kuliner,type_clean_Minat Khusus,type_clean_Museum,type_clean_Pantai,type_clean_Pendidikan,type_clean_Religi,type_clean_Seni,type_clean_Wisata Air
0,9,Candi Borobudur,4.7,81922,50000.0,50000.0,-7.607087,110.203623,Candi yang pernah masuk sebagai salah satu dar...,0,...,1,0,0,0,0,0,0,0,0,0
1,10,Candi Prambanan,4.7,71751,50000.0,50000.0,-7.751835,110.491532,Candi Prambanan adalah kompleks candi Hindu te...,0,...,1,0,0,0,0,0,0,0,0,0
2,24,Tebing Breksi,4.4,51431,10000.0,10000.0,-7.781477,110.504576,Tebing Breksi merupakan tempat wisata yang ber...,0,...,0,0,0,0,0,0,0,0,0,0
3,343,Gembira Loka Zoo,4.5,36337,20000.0,25000.0,-7.806234,110.396798,Gambira Loka adalah kebun binatang yang berada...,0,...,0,0,0,0,0,0,0,0,0,0
4,346,The Palace of Yogyakarta (Keraton Yogyakarta),4.6,30091,8000.0,8000.0,-7.805284,110.364203,Kompleks keraton merupakan museum yang menyimp...,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,139,Pemancingan Adi Winata,5.0,1,0.0,0.0,-7.704577,110.512011,Tempat rekreasi keluarga yang menawarkan fasil...,0,...,0,0,0,0,0,0,0,0,0,1
472,105,Ruang Perawatan Jenderal Soedirman,5.0,1,4000.0,4000.0,-7.776474,110.376744,Situs sejarah berupa ruangan tempat Jenderal S...,0,...,1,0,0,0,0,0,0,0,0,0
473,110,Situs Gedong Pusoko,5.0,1,15000.0,15000.0,-7.807846,110.403758,Situs arkeologi yang menyimpan tinggalan sejar...,0,...,1,0,0,0,0,0,0,0,0,0
474,164,Taman Edukasi dan Outbound Sunan Kalijaga,5.0,1,0.0,0.0,-7.809207,110.413252,Area edukatif yang menyediakan kegiatan outbou...,0,...,0,0,0,0,0,0,0,0,0,0


## Clean Text Description

In [ ]:
df_description = df_encoded.copy()

# Salin kolom description dan ubah ke lowercase
df_description['description_clean'] = df_description['description'].astype(str).str.lower()

# Bersihkan teks: hilangkan tanda baca, angka, dan karakter khusus
df_description['description_clean'] = df_description['description_clean'].apply(
    lambda x: re.sub(f"[{re.escape(string.punctuation)}0-9]", " ", x)
)

# Hilangkan whitespace berlebih
df_description['description_clean'] = df_description['description_clean'].apply(lambda x: re.sub(r"\s+", " ", x).strip())

# Tampilkan hasil
df_description


,no,nama,vote_average,vote_count,htm_weekday,htm_weekend,latitude,longitude,description,type_clean_Agrowisata,...,type_clean_Desa Wisata,type_clean_Kuliner,type_clean_Minat Khusus,type_clean_Museum,type_clean_Pantai,type_clean_Pendidikan,type_clean_Religi,type_clean_Seni,type_clean_Wisata Air,description_clean
0,9,Candi Borobudur,4.7,81922,50000.0,50000.0,-7.607087,110.203623,Candi yang pernah masuk sebagai salah satu dar...,0,...,0,0,0,0,0,0,0,0,0,candi yang pernah masuk sebagai salah satu dar...
1,10,Candi Prambanan,4.7,71751,50000.0,50000.0,-7.751835,110.491532,Candi Prambanan adalah kompleks candi Hindu te...,0,...,0,0,0,0,0,0,0,0,0,candi prambanan adalah kompleks candi hindu te...
2,24,Tebing Breksi,4.4,51431,10000.0,10000.0,-7.781477,110.504576,Tebing Breksi merupakan tempat wisata yang ber...,0,...,0,0,0,0,0,0,0,0,0,tebing breksi merupakan tempat wisata yang ber...
3,343,Gembira Loka Zoo,4.5,36337,20000.0,25000.0,-7.806234,110.396798,Gambira Loka adalah kebun binatang yang berada...,0,...,0,0,0,0,0,0,0,0,0,gambira loka adalah kebun binatang yang berada...
4,346,The Palace of Yogyakarta (Keraton Yogyakarta),4.6,30091,8000.0,8000.0,-7.805284,110.364203,Kompleks keraton merupakan museum yang menyimp...,0,...,0,0,0,0,0,0,0,0,0,kompleks keraton merupakan museum yang menyimp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,139,Pemancingan Adi Winata,5.0,1,0.0,0.0,-7.704577,110.512011,Tempat rekreasi keluarga yang menawarkan fasil...,0,...,0,0,0,0,0,0,0,0,1,tempat rekreasi keluarga yang menawarkan fasil...
472,105,Ruang Perawatan Jenderal Soedirman,5.0,1,4000.0,4000.0,-7.776474,110.376744,Situs sejarah berupa ruangan tempat Jenderal S...,0,...,0,0,0,0,0,0,0,0,0,situs sejarah berupa ruangan tempat jenderal s...
473,110,Situs Gedong Pusoko,5.0,1,15000.0,15000.0,-7.807846,110.403758,Situs arkeologi yang menyimpan tinggalan sejar...,0,...,0,0,0,0,0,0,0,0,0,situs arkeologi yang menyimpan tinggalan sejar...
474,164,Taman Edukasi dan Outbound Sunan Kalijaga,5.0,1,0.0,0.0,-7.809207,110.413252,Area edukatif yang menyediakan kegiatan outbou...,0,...,0,0,0,0,0,0,0,0,0,area edukatif yang menyediakan kegiatan outbou...


In [ ]:
df_description.drop('description', axis=1, inplace=True)

In [ ]:
df_final = df_description.copy()
df_final

,no,nama,vote_average,vote_count,htm_weekday,htm_weekend,latitude,longitude,type_clean_Agrowisata,type_clean_Alam,...,type_clean_Desa Wisata,type_clean_Kuliner,type_clean_Minat Khusus,type_clean_Museum,type_clean_Pantai,type_clean_Pendidikan,type_clean_Religi,type_clean_Seni,type_clean_Wisata Air,description_clean
0,9,Candi Borobudur,4.7,81922,50000.0,50000.0,-7.607087,110.203623,0,0,...,0,0,0,0,0,0,0,0,0,candi yang pernah masuk sebagai salah satu dar...
1,10,Candi Prambanan,4.7,71751,50000.0,50000.0,-7.751835,110.491532,0,0,...,0,0,0,0,0,0,0,0,0,candi prambanan adalah kompleks candi hindu te...
2,24,Tebing Breksi,4.4,51431,10000.0,10000.0,-7.781477,110.504576,0,1,...,0,0,0,0,0,0,0,0,0,tebing breksi merupakan tempat wisata yang ber...
3,343,Gembira Loka Zoo,4.5,36337,20000.0,25000.0,-7.806234,110.396798,0,0,...,0,0,0,0,0,0,0,0,0,gambira loka adalah kebun binatang yang berada...
4,346,The Palace of Yogyakarta (Keraton Yogyakarta),4.6,30091,8000.0,8000.0,-7.805284,110.364203,0,0,...,0,0,0,0,0,0,0,0,0,kompleks keraton merupakan museum yang menyimp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,139,Pemancingan Adi Winata,5.0,1,0.0,0.0,-7.704577,110.512011,0,0,...,0,0,0,0,0,0,0,0,1,tempat rekreasi keluarga yang menawarkan fasil...
472,105,Ruang Perawatan Jenderal Soedirman,5.0,1,4000.0,4000.0,-7.776474,110.376744,0,0,...,0,0,0,0,0,0,0,0,0,situs sejarah berupa ruangan tempat jenderal s...
473,110,Situs Gedong Pusoko,5.0,1,15000.0,15000.0,-7.807846,110.403758,0,0,...,0,0,0,0,0,0,0,0,0,situs arkeologi yang menyimpan tinggalan sejar...
474,164,Taman Edukasi dan Outbound Sunan Kalijaga,5.0,1,0.0,0.0,-7.809207,110.413252,0,0,...,0,0,0,0,0,0,0,0,0,area edukatif yang menyediakan kegiatan outbou...


## Output Pre-Processing

In [ ]:
df_final.to_csv('dataset_wisata_jogja_preprocessed.csv', index=False)

In [ ]:
from google.colab import files
files.download('dataset_wisata_jogja_preprocessed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>